In [ ]:
import psycopg2
import pandas as pd
from getpass import getpass
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import re 
import pandas as pd
from tqdm import tqdm


user = input("Bitte User für DB eingeben")
pswd = getpass("Bitte Passwort für DB eingeben")
conn = psycopg2.connect(f"dbname='orent001_spotify_test' user='{user}' host='localhost' port='9001' password='{pswd}'")


In [ ]:
querry = """SELECT playlist.playlist_id, playlist.name, playlist.collaborative, playlist.modified_at, playlist.num_tracks, playlist.num_albums, playlist.num_followers, playlist.duration_ms as duration_ms_playlist, playlist.num_artists, playlist.num_edits,
            p_enthaelt_s.pos,
            song.song_id, song.interpret, song.album, song.track_name, song.duration_ms as duration_ms_song,
            interpret.artist_name
from playlist
INNER JOIN p_enthaelt_s on p_enthaelt_s.playlist = playlist.playlist_id
INNER JOIN song on song_id = p_enthaelt_s.song
INNER JOIN interpret on interpret_id = song.interpret
"""
dataset = pd.io.sql.read_sql_query(querry,conn)
dataset

In [ ]:
dataset.to_csv("data.csv")

In [ ]:
def getFeatures(index):

    cid = 'cf84ae5a48054fffb8b47f6b9239748c'
    secret = 'c74bbff82dcb4ddbba73f6e3aa7726be'


    client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
    sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)
    
    #Audio features
    features = sp.audio_features(index)[0]
    
    #Artist of the track, for genres and popularity
    artist = sp.track(index)["artists"][0]["id"]
    artist_pop = sp.artist(artist)["popularity"]
    artist_genres = sp.artist(artist)["genres"]
    
    #Track popularity
    track_pop = sp.track(index)["popularity"]
    
    #Add in extra features
    features["artist_pop"] = artist_pop
    if artist_genres:
        features["genres"] = " ".join([re.sub(' ','_',i) for i in artist_genres])
    else:
        features["genres"] = "unknown"
    features["track_pop"] = track_pop
    
    return features

In [ ]:
dataPath = 'data.csv'
df = pd.read_csv(dataPath)
df.head()

In [ ]:
testDF = df

In [ ]:
first = df["song_id"].unique()[:10000]
second = df["song_id"].unique()[10000:20000]
third = df["song_id"].unique()[20000:]
dataList = [first,second,third]

In [ ]:
featureList = []

for i in tqdm([uri for uri in dataList[0]]):
    try:
        featureList.append(getFeatures(i))
    except:
        continue

In [ ]:
for i in tqdm([uri for uri in dataList[1]]):
    try:
        featureList.append(getFeatures(i))
    except:
        continue

In [ ]:
for i in tqdm([uri for uri in dataList[2]]):
    try:
        featureList.append(getFeatures(i))
    except:
        continue

In [ ]:
featureDF = pd.DataFrame(featureList)
featureDF

In [ ]:
new_df = pd.merge(testDF,featureDF, left_on = "song_id", right_on= "id")
new_df.to_csv('combined.csv')